# **Optimal Product Selection:**

Problem Source: F. S. Hillier and G. J. Lieberman, "Introduction to Mathematical Programming," 2nd Edition, McGraw-Hill Inc., New York, 1995.

**Problem Statement:**

A toy manufacturer has developed two new toys for possible inclusion in its product line for the upcoming Christmas season. Setting up the production facilities to begin production would cost $ \$50,000$ for toy $1$ and $ \$80,000$ for toy $2$. Once these costs are covered, the toys would generate a unit profit of $ \$10$ for toy $1$ and $ \$15$ for toy $2$.

The company has two factories that are capable of producing these toys. However, to avoid doubling the start-up costs, just one factory would be used and the choice would be based on maximizing profit. For administrative reasons, the same factory would be used for both new toys if both are produced.

Toy $1$ can be produced at the rate of $50$ per hour in factory $1$ and $40$ per hour in factory $2$. Toy $2$ can be produced at the rate of $40$ per hour in factory $1$ and $25$ per hour in factory $2$. Factories $1$ and $2$, respectively, have $500$ and $700$ hours of production time available before Christmas that could be used to produce these toys.

It is not known whether these two toys would be continued after Christmas. Therefore, the problem is to determine how many units (if any) of each new toy should be produced before Christmas in order to maximize the total profit.

**Modeling:**

$x_{i} \qquad  i\in {1,2} \quad $   continuous variables for number of units of Toys produced

$y_{j} \qquad  j\in {1,2} \quad $   binary variable if Factory j is chosen for production

$\\ $

$Objective\ Function:\\ $

$Max\ Z = 10x_{1}+15x_{2}-50000y_{1}-80000y_{2}$

$\\ $

$Constraints:\\ $

Only one factory can be chosen:

$y_{1}+y_{2}=1$

Production time constraints: Each factory has a limited number of hours available, and the number of units produced for each toy depends on the rate at which each factory can produce them.

$\frac{x_{1}}{50} + \frac{x_{2}}{40} \leq 500y_{1}$

$\frac{x_{1}}{40} + \frac{x_{2}}{25} \leq 700y_{1}$

Non-negativity and Binary Constraints:

$x_{1},x_{2} \geq 0$

$y_{1},y_{2} \in {0,1}$

## Using Pulp Library

In [1]:
!pip install pulp
from pulp import *
import numpy as np
import pandas as pd

In [2]:
#Defining Variables

x = LpVariable.dicts('x', ((i) for i in range(1,3)), lowBound = 0, cat='Continuous')
y = LpVariable.dicts('y', ((i) for i in range(1,3)), cat='Binary')

In [3]:
#Defining Objective Function

model = LpProblem('model', LpMaximize)

model += 10*x[1]+15*x[2]-50000*y[1]-80000*y[2]

In [4]:
#Constraints

#Only one factory can be chosen: Since only one factory can be used to produce the toys, we impose the following constraint:
model += y[1]+y[2] == 1

#Production time constraints: Each factory has a limited number of hours available, and the number of units produced for each toy depends on the rate at which each factory can produce them.
model += x[1]*(1/50) + x[2]*(1/40) <= 500*y[1]
model += x[1]*(1/40) + x[2]*(1/25) <= 700*y[1]

In [5]:
#Model summary

model

model:
MAXIMIZE
10*x_1 + 15*x_2 + -50000*y_1 + -80000*y_2 + 0
SUBJECT TO
_C1: y_1 + y_2 = 1

_C2: 0.02 x_1 + 0.025 x_2 - 500 y_1 <= 0

_C3: 0.025 x_1 + 0.04 x_2 - 700 y_1 <= 0

VARIABLES
x_1 Continuous
x_2 Continuous
0 <= y_1 <= 1 Integer
0 <= y_2 <= 1 Integer

In [6]:
#Solving model

model.solve()
print('Optimal Solution:', pulp.value(model.objective))

Optimal Solution: 221428.56900000002


In [7]:
for variables in model.variables():
  print(f'{variables.name}:{variables.varValue}')

x_1:14285.714
x_2:8571.4286
y_1:1.0
y_2:0.0


In [8]:
VNames = []
for variables in model.variables():
  VNames.append(variables.name)

VValue = []
for variables in model.variables():
  VValue.append(np.round(variables.varValue, 2))

data = {'Variables': VNames, 'Value': VValue}
pd.DataFrame(data, index = range(1, len(VValue)+1))

,Variables,Value
1,x_1,14285.71
2,x_2,8571.43
3,y_1,1.00
4,y_2,0.00


In [9]:
print('Current Status: ', LpStatus[model.status])

Current Status:  Optimal


## Using Pyomo Library

In [10]:
!pip install pyomo
!apt-get install -y -qq glpk-utils

from pyomo.environ import *

In [11]:
model = ConcreteModel()

model.indicesx = Set(initialize=list(range(1,3)))
model.indicesy = Set(initialize=list(range(1,3)))
model.x = Var(model.indicesx, domain=NonNegativeReals)
model.y = Var(model.indicesy, domain=Binary)

model.obj = Objective(expr=10*model.x[1]+15*model.x[2]-50000*model.y[1]-80000*model.y[2], sense=maximize)

model.constraint1 = Constraint(expr=model.y[1]+model.y[2] == 1)
model.constraint2 = Constraint(expr=model.x[1]*(1/50) + model.x[2]*(1/40) <= 500*model.y[1])
model.constraint3 = Constraint(expr=model.x[1]*(1/40) + model.x[2]*(1/25) <= 700*model.y[1])

SolverFactory('glpk').solve(model)

model.display()

Model unknown

  Variables:
    x : Size=2, Index=indicesx
        Key : Lower : Value            : Upper : Fixed : Stale : Domain
          1 :     0 : 14285.7142857143 :  None : False : False : NonNegativeReals
          2 :     0 : 8571.42857142857 :  None : False : False : NonNegativeReals
    y : Size=2, Index=indicesy
        Key : Lower : Value : Upper : Fixed : Stale : Domain
          1 :     0 :   1.0 :     1 : False : False : Binary
          2 :     0 :   0.0 :     1 : False : False : Binary

  Objectives:
    obj : Size=1, Index=None, Active=True
        Key  : Active : Value
        None :   True : 221428.5714285716

  Constraints:
    constraint1 : Size=1
        Key  : Lower : Body : Upper
        None :   1.0 :  1.0 :   1.0
    constraint2 : Size=1
        Key  : Lower : Body                   : Upper
        None :  None : 2.8421709430404007e-13 :   0.0
    constraint3 : Size=1
        Key  : Lower : Body                  : Upper
        None :  None : 4.5474735088646

In [12]:
print("Variable values:")
model.x.pprint()
model.y.pprint()

Variable values:
x : Size=2, Index=indicesx
    Key : Lower : Value            : Upper : Fixed : Stale : Domain
      1 :     0 : 14285.7142857143 :  None : False : False : NonNegativeReals
      2 :     0 : 8571.42857142857 :  None : False : False : NonNegativeReals
y : Size=2, Index=indicesy
    Key : Lower : Value : Upper : Fixed : Stale : Domain
      1 :     0 :   1.0 :     1 : False : False : Binary
      2 :     0 :   0.0 :     1 : False : False : Binary
